In [ ]:
#| label: mtsatFigB3jn

# Prepare Python environment

import scipy.io as sio
import numpy as np
from pathlib import Path

from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
data_dir = dataset_path / "06-MT-03-MTsat" / "06-MT-03-MTsat" 
data_file = "sim2.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

Mz_after = mat_contents["Mz_after"][0]
delta_Mz_T1relax = mat_contents["delta_Mz_T1relax"][0]
Mz_before = mat_contents["Mz_before"][0]
M0_remainingTR_free =  mat_contents["M0_remainingTR_free"][0]
delta_Mz_T1relax_remaining =  mat_contents["delta_Mz_T1relax_remaining"][0]

x=np.arange(1,len(Mz_before)+1)
y1 = (1-(Mz_after-delta_Mz_T1relax)/Mz_before)*100
y2=(1-(M0_remainingTR_free-delta_Mz_T1relax_remaining)/Mz_before)*100
y3=(1-(Mz_after-delta_Mz_T1relax)/Mz_before)*100+(1-(M0_remainingTR_free-delta_Mz_T1relax_remaining)/Mz_before)*100

# Plot Figure 1

# Module imports

import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.subplots import make_subplots

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

# PYTHON CODE

init_notebook_mode(connected=True)
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.

## Setup for plots
fig = make_subplots(rows=1, cols=1, horizontal_spacing = 0.1)

fig.add_trace(
        go.Scatter(
        x=x,
        y=y1,
        hoverinfo = 'y',
        visible=True,
        name='MTsat contribution from MT pulse event'
        )
    )

fig.add_trace(
        go.Scatter(
        x=x,
        y=y2,
        hoverinfo = 'y',
        visible=True,
        name='MTsat contribution from cross-relaxation event'
        )
    )

fig.add_trace(
        go.Scatter(
        x=x,
        y=y3,
        hoverinfo = 'y',
        visible=True,
        name='Total MTsat for TR'
        )
    )

layout = go.Layout(
    width=600,
    height=500,
    margin=go.layout.Margin(
        l=60,
        r=30,
        b=60,
        t=60,
    ),
    legend={
        "x": 0.3,
        "y": 0.9,
        "xref": "paper",
        "yref": "paper",
    },
    xaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2,
        title='Repetition #'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2,
        title="\"MTsat\" (%)"
    ),
)

fig.update_layout(layout)

iplot(fig, filename = 'fig4.html', config = config)
